# Boolean Query

In this notebook, we will implement a Boolean information retrieval model by Elasticsearch.  
We have these steps: <br>

**1- Connect to the Elasticsearch Cluster and Create an Index <br>
2- Indexing Documents <br>
3- Boolean Retrieval (Fill where ever it says #TODO in this part)** <br>

Let me know if you have any problems with this notebook or implementation. <br>
**Telegram**: [@Mohammad_Ardestani4](https://t.me/Mohammad_Ardestani4) <br>
**Email**: mjavad.ardestani00@gmial.com

**Student Information:**
AmirMohammad Babaei
9831011

In [3]:
!pip install elasticsearch
!pip install tqdm

  Using cached tqdm-4.64.0-py2.py3-none-any.whl (78 kB)


In [4]:
from elasticsearch import Elasticsearch, helpers
import os, json, time
from datetime import datetime
from tqdm import tqdm 

## Ignore The Security Warnings
Here we ignore the security warnings. It's suggested that you don't run below cell until you ensure that all warnings are about security. 


In [3]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
!echo -e "[ELASTIC]\ncloud_id = IR_phase3:dXMtY2VudHJhbDEuZ2NwLmNsb3VkLmVzLmlvJDE4MDM5N2JjM2YyMTRjMzVhYzFmMjgzYjNmM2ZkYzk1JDYwNmE4YzY5ZmM5MjQ0ZGQ4MGRhZDdhODk1OTI0YzNl\nuser = elastic\npassword = Dq9uLcCHKbnsqXMQtASef5z0\n" > example.ini

In [6]:
!cat example.ini

[ELASTIC]
cloud_id = IR_phase3:dXMtY2VudHJhbDEuZ2NwLmNsb3VkLmVzLmlvJDE4MDM5N2JjM2YyMTRjMzVhYzFmMjgzYjNmM2ZkYzk1JDYwNmE4YzY5ZmM5MjQ0ZGQ4MGRhZDdhODk1OTI0YzNl
user = elastic
password = Dq9uLcCHKbnsqXMQtASef5z0



In [7]:
import json
dataset_path = 'IR_data_news_12k.json'
with open(dataset_path) as f:
    data = json.load(f)
print(data['0'].keys())

dict_keys(['title', 'content', 'tags', 'date', 'url', 'category'])


## Connect to the Elasticsearch Cluster and Create an Index
After starting your Elasticsearch on your pc (localhost:9200 is the default), we have to connect to it via the following piece of code.


In [8]:
import configparser

config = configparser.ConfigParser()
config.read('example.ini')

['example.ini']

In [9]:
config['ELASTIC']

<Section: ELASTIC>

In [ ]:
index_name = 'ta3_index'
es = Elasticsearch(
    cloud_id=config['ELASTIC']['cloud_id'],
    http_auth=(config['ELASTIC']['user'], config['ELASTIC']['password'])
)
es.indices.create(index = index_name)

### Check the Cluster Status

In [11]:
dict(es.info())

{'name': 'instance-0000000001',
 'cluster_name': '180397bc3f214c35ac1f283b3f3fdc95',
 'cluster_uuid': 'pJv6cAu-S_KeEz6j-dOqGA',
 'version': {'number': '8.3.0',
  'build_type': 'docker',
  'build_hash': '5b8b981647acdf1ba1d88751646b49d1b461b4cc',
  'build_date': '2022-06-23T22:48:49.607492124Z',
  'build_snapshot': False,
  'lucene_version': '9.2.0',
  'minimum_wire_compatibility_version': '7.17.0',
  'minimum_index_compatibility_version': '7.0.0'},
 'tagline': 'You Know, for Search'}

## Indexing Documents 

we can add our documents to created index in two ways:<br>
1- one by one in for loop. <br>
2- use [Bulk API](https://www.elastic.co/guide/en/elasticsearch/reference/current/docs-bulk.html#:~:text=Bulk%20APIedit,can%20greatly%20increase%20indexing%20speed.), which performs multiple indexing or delete operations in a single API call. This reduces overhead and can significantly increase indexing speed.

###  for loop by es.index 
**You dont have to run this cell**. 

In [ ]:
for i in tqdm(range(len(data))):
    es.index(index = index_name, id=i, document=data[str(i)])
    i = i + 1

100%|█████████████████████████████████████| 12202/12202 [03:41<00:00, 55.02it/s]


###  Bulk API

In [12]:
from elasticsearch.helpers import bulk

def bulk_sync():
    actions = [
        {
            '_index': 'ta3_index',
            '_id':doc_id,
            '_source': doc
        } for doc_id,doc in data.items()
    ]
    bulk(es, actions)


In [13]:
start = time.time()
bulk_sync()
end = time.time()
print("Indexing all documents took about {:.2f} seconds".format(end - start))

Indexing all documents took about 91.39 seconds


### check index

In [38]:
es.count(index = 'ta3_index')

ObjectApiResponse({'count': 12202, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}})

## Boolean Retrieval 

### Bool Query Structure

It is built using one or more boolean clauses, each clause with a typed occurrence. The occurrence types are:

**must**: The clause (query) must appear in matching documents and will contribute to the score.
<br><br>
**filter**: The clause (query) must appear in matching documents. However unlike must the score of the query will be ignored. Filter clauses are executed in filter context, meaning that scoring is ignored and clauses are considered for caching.
<br><br>
**should**: The clause (query) should appear in the matching document.
<br><br>
**must_not**: The clause (query) must not appear in the matching documents. Clauses are executed in filter context meaning that scoring is ignored and clauses are considered for caching. Because scoring is ignored, a score of 0 for all documents is returned.

For further information, you can read this [Document](https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-bool-query.html). 

###  <span style="color:red"> TODO </span>
You should read about [match query](https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-match-query.html#match-top-level-params) and [match phrase query](https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-match-query-phrase.html) then complete the below cell.

In [28]:
query= {
        "bool": {
          "should": [
              { 
                  #TODO: add a match query structure ==> use for normal words
                  "match": {
                      "content": {
                          "query": "آمریکا"
                      }
                  }
              }, 
              
              { 
                  #TODO: add a match pharse query strucutre ==> use for words in <"">
                  "match_phrase": {
                      "content": "تحریم هسته‌ای"
                  }
              },
   
          ],
          "must_not": [
              {
                  #TODO: add a match or match pharse query structure ==> use for words after <!>
                  "match": {
                      "content": {
                          "query": "ایران"
                      }
                  }
              }
          ],
        },
    }

In [33]:
query2= {
        "bool": {
          "should": [
              { 
                  #TODO: add a match query structure ==> use for normal words
                  "match": {
                      "content": {
                          "query": "اورشلیم"
                      }
                  }
              }, 
   
          ],
          "must_not": [
              {
                  #TODO: add a match or match pharse query structure ==> use for words after <!>
                  "match": {
                      "content": {
                          "query": "صهیونیست"
                      }
                  }
              }
          ],
        },
    }

### Search query

The _source field contains the original JSON document body that was passed at index time. The _source field itself is not indexed (and thus is not searchable), but it is stored so that it can be returned when executing fetch requests, like get or search.

For further information, you can read this [Document](https://www.elastic.co/guide/en/elasticsearch/reference/current/search-search.html#search-api-response-body).

In [29]:
res = es.search(index = 'ta3_index', query=query, _source= ["url"])
res = dict(res)

In [34]:
res2 = es.search(index = 'ta3_index', query=query2, _source= ["url"])
res2 = dict(res2)

### Results

In [31]:
def print_result(res):
    print("{} results in {} s: ".format(res['hits']['total']['value'] ,res['took']/1000))
    for doc in res['hits']['hits']:
        print(doc['_source']['url'])

In [32]:
print_result(res)

171 results in 0.007 s: 
https://www.farsnews.ir/news/14001214000789/نماینده-کلیمیان-در-مجلس-دهم--کارتل‌ها-بر-تصمیمات-هیأت-حاکمه-آمریکا
https://www.farsnews.ir/news/14001217000665/خباثت‌های-آمریکا-در-برجام-روسیه-و-چین-را-هم-به-این-کشور-بدبین-کرد
https://www.farsnews.ir/news/14001213000089/آمریکا-با-بحران‌آفرینی-به-حیات-خود-ادامه-می-دهد
https://www.farsnews.ir/news/14001211000321/آمریکا-دولت‌های-متحد-خود-را-در-زمان-اضطرار-تنها-می‌گذارد
https://www.farsnews.ir/news/14001211000898/سود-مافیای-اسلحه‌سازی-آمریکا-در-ناامن-بودن-جهان-است
https://www.farsnews.ir/news/14001214000825/آمریکا-رژیمی-مافیایی-است-و-مردم-در-تصمیمات-حاکمان-آن-جایگاهی-ندارند
https://www.farsnews.ir/news/14001212000725/آمریکا-برای-فروش-تسلیحات-خود-به-ایجادناامنی-و-بحران‌آفرینی-نیاز-دارد
https://www.farsnews.ir/news/14001204000444/۲۸-آمریکایی-در-جام-یاشاردوغو-جردن-باروز-هم-می‌آید
https://www.farsnews.ir/news/14001213000328/کارتل‌های-اقتصادی-رؤسای-جمهور-آمریکا-را-تعیین-می‌کنند
https://www.farsnews.ir/news/14001211000220/نمای

![image.png](./images/america-1.png)

![image.png](./images/america-2.png)

![image.png](./images/america-3.png)

<div dir="rtl">
    همانطور که در سه تصویر بالا مشاهده می شود، در هیچ یک از این داک ها کلمه ایران وجود ندارد. همچنین باید توجه داشت که در این مدل به دلیل این که هیچ کلمه ای با عبارت «تحریم هسته‌ای» وجود نداشته است، داک هایی را برگردانده است که دارای تعداد کلمه آمریکا بیشتری بوده اند.
</div>

In [35]:
print_result(res2)

0 results in 0.002 s: 


<div dir="rtl">
    در این حالت نیز همانند مدل قبلی به دلیل نبودن داکی با این ویژگی، در نتیجه هیچ داکی برگردانده نشده است.
</div>

<div dir="rtl">
    به عنوان کاربر نهایی، استفاده از این مدل جدید را ترجیح می دهم. به دلیل سرعت بالاتر و امکانات بیشتر این مدل.
</div>